In [74]:
from langgraph.graph import StateGraph,START,END
from typing import TypedDict,List,Annotated,Literal
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from pydantic import BaseModel,Field
import operator
from langgraph.checkpoint.memory import InMemorySaver

load_dotenv()
model = ChatGoogleGenerativeAI(model='gemini-1.5-flash-latest')

In [75]:
class Marks(BaseModel):
    marks : int = Field(...,description='Give marks to the joke based on how enjoyble it is ')

In [76]:
structure_model = model.with_structured_output(Marks)

type(structure_model.invoke('my name is Jugal').marks)

int

In [77]:
class JokeState(TypedDict):
    
    title : str
    joke : str
    marks : str

In [78]:
def create_joke(state : JokeState):
    topic = state['title']
    prompt = f"Create a Joke on this Topic : {topic}"
    joke = model.invoke(prompt).content
    return {'joke' : joke}

def give_marks(state : JokeState):
    joke = state['joke']
    score = structure_model.invoke(joke).marks
    return {'marks' : score}

In [79]:
graph = StateGraph(JokeState)

graph.add_node('create_joke',create_joke)
graph.add_node('give_marks',give_marks)

graph.add_edge(START,'create_joke')
graph.add_edge('create_joke','give_marks')
graph.add_edge('give_marks',END)

checkpointer = InMemorySaver()

In [80]:
workflow = graph.compile(checkpointer=checkpointer)

In [81]:
config1 = {"configurable": {"thread_id": "1"}}

In [90]:
workflow.invoke({'title':'python'}, config=config1)

{'title': 'python',
 'joke': 'Why did the Python program go to therapy?\n\nBecause it had too many bugs and needed some serious `debug`ging!',
 'marks': 5}

In [91]:
print(workflow.get_state(config1))

StateSnapshot(values={'title': 'python', 'joke': 'Why did the Python program go to therapy?\n\nBecause it had too many bugs and needed some serious `debug`ging!', 'marks': 5}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f079f6b-4c20-6282-800a-0826f272d591'}}, metadata={'source': 'loop', 'step': 10, 'parents': {}}, created_at='2025-08-15T16:41:35.001459+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f079f6b-12db-6265-8009-ce026e92f5e9'}}, tasks=(), interrupts=())


In [92]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'title': 'python', 'joke': 'Why did the Python program go to therapy?\n\nBecause it had too many bugs and needed some serious `debug`ging!', 'marks': 5}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f079f6b-4c20-6282-800a-0826f272d591'}}, metadata={'source': 'loop', 'step': 10, 'parents': {}}, created_at='2025-08-15T16:41:35.001459+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f079f6b-12db-6265-8009-ce026e92f5e9'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'title': 'python', 'joke': 'Why did the Python program go to therapy?\n\nBecause it had too many bugs and needed some serious `debug`ging!', 'marks': 5}, next=('give_marks',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f079f6b-12db-6265-8009-ce026e92f5e9'}}, metadata={'source': 'loop', 'step': 9, 'parents': {}}, created_at='2025-08-15T16:41:28.996310+00:00', parent_c

In [94]:
workflow.get_state({'configurable': {'thread_id':'1','checkpoint_id': '1f079f68-fe4a-62b0-8003-65f26bed81bb'}})

StateSnapshot(values={'title': 'pizza', 'joke': "Why does the pizza slice like the mushroom so much?\n\nBecause he's a fungi!", 'marks': 5}, next=('__start__',), config={'configurable': {'thread_id': '1', 'checkpoint_id': '1f079f68-fe4a-62b0-8003-65f26bed81bb'}}, metadata={'source': 'input', 'step': 3, 'parents': {}}, created_at='2025-08-15T16:40:33.152683+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f079f5d-faf1-6555-8002-2cbfba499a80'}}, tasks=(PregelTask(id='482beb26-22b3-febe-7cca-37408a8ca410', name='__start__', path=('__pregel_pull', '__start__'), error=None, interrupts=(), state=None, result={'title': 'pizza'}),), interrupts=())

In [95]:
workflow.invoke(None,{'configurable': {'thread_id':'1','checkpoint_id': '1f079f68-fe4a-62b0-8003-65f26bed81bb'}})


{'title': 'pizza',
 'joke': "Why does the pizza slice like the mushroom so much?\n\nBecause he's a fungi!",
 'marks': 5}